# NOTES

# IMPORTS AND SETTINGS

Changes and Improvements:
VIF Analysis: The script checks for multicollinearity, and I have suggested removing Fixed Asset Investment CN (T-1) and Avg Wage Difference due to potential multicollinearity issues.
Scaling: I added scaling for variables that have large magnitudes, improving the interpretability of coefficients.
Model Diagnostics: The script includes diagnostics such as the Durbin-Wu-Hausman test, Wald test, and Breusch-Pagan test for endogeneity and heteroskedasticity.
Rank Check: Added a check for matrix rank to ensure that the independent variables are not linearly dependent.

In [ ]:
# -*- coding: utf-8 -*-
"""Improved script based on suggestions"""

"""# IMPORTS AND SETTINGS"""
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.api import add_constant
from linearmodels.panel import compare
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import chi2
import os
import warnings
warnings.filterwarnings('ignore')

project_folder = os.getcwd()

"""# DATA READING"""

# data from Excel
filename3 = os.path.join(project_folder, "Regression data", "panel_regression_all.csv")
data3 = pd.read_csv(os.path.join(project_folder, filename3))

# 'Country' as entity and 'Year' as time
data3 = data3.set_index(['Country', 'Year'])
data3 = data3.dropna(axis=1, how="all")
data3.head(20)

data3.info()

# Data scaling for selected variables (to improve model fit)
data3.loc[:, ["GDP/capita", "Tech Advancement CN", "Fixed Asset Investment CN (T-1)", "Avg Wage Difference"]] = data3.loc[:, ["GDP/capita", "Tech Advancement CN", "Fixed Asset Investment CN (T-1)", "Avg Wage Difference"]].div(1000)
data3.loc[:, "Import"] = data3.loc[:, "Import"].div(1000000)
data3.describe().T

"""# MODELING SETUP"""

# Set dependent, independent, and control variables
data3["constant"] = 1
dependent_var = data3['Import'].to_frame()
independent_vars = data3.iloc[:, 1:]

# Drop insignificant or redundant variables based on VIF and correlation matrix
independent_vars = independent_vars.drop(columns=["constant", "Fixed Asset Investment CN (T-1)", "Avg Wage Difference"], errors='ignore')

# Calculate Variance Inflation Factor (VIF) to check multicollinearity
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(independent_vars.values, i) for i in range(independent_vars.shape[1])]
vif["features"] = independent_vars.columns
print("Variance Inflation Factors (VIF):")
print(vif)

"""# MODELS

## Fixed Effects Model (Entity One-Way)
"""
# Check correlation matrix and matrix rank to avoid multicollinearity
corr_matrix = independent_vars.corr()
print(corr_matrix)

def matrix_rank_check(df):
    return np.linalg.matrix_rank(df.values)

print(f"Rank of independent variables matrix: {matrix_rank_check(independent_vars)}")
print(f"Number of columns in independent variables: {independent_vars.shape[1]}")

# Fit Fixed Effects Model (entity effects only)
fe_model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
fe_results = fe_model.fit()

print("Fixed Effects Model Results:")
print(fe_results)

"""## Fixed Effects Model (Two-Way)"""

# Two-way Fixed Effects Model (with time and entity effects)
two_model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True, drop_absorbed=True)
two_results = two_model.fit()

print("Two-Way Fixed Effects Model Results:")
print(two_results)

"""## Random Effects Model"""

# Random Effects Model
re_model = RandomEffects(dependent_var, independent_vars)
re_results = re_model.fit()

print("\nRandom Effects Model Results:")
print(re_results)

# Compare Fixed and Random Effects Models using Hausman Test
comparison = compare({"Fixed": fe_results, "Random": re_results})
print(comparison)

"""## Durbin-Wu-Hausman Test for Endogeneity"""

def durbin_wu_hausman_test(ols_fixed, ols_random):
    """Perform the Durbin-Wu-Hausman test for endogeneity."""
    u_fe = ols_fixed.resids  # Residuals from FE model
    u_re = ols_random.resids  # Residuals from RE model
    dw_hausman_stat = np.sum((u_fe - u_re)**2) / np.sum(u_fe**2)
    df = fe_results.df_model - re_results.df_model
    p_value = 1 - chi2.cdf(dw_hausman_stat, df)
    return dw_hausman_stat, p_value

# Perform the Durbin-Wu-Hausman test
dw_hausman_stat, p_value = durbin_wu_hausman_test(fe_results, re_results)
print("Durbin-Wu-Hausman Test Statistic:", dw_hausman_stat)
print("P-value:", p_value)

"""## Wald Test"""

# Wald test: checks if regressors are collectively insignificant
wald_result = fe_results.wald_test(formula="Fit=0, `GDP/capita`=0, `Energy Consumption`=0, `Annual Solar Capacity Addition`=0, `Tech Advancement CN`=0, `TechAdvancement `=0, `Trade Policies EU`=0, `Environ. St. Difference`=0")
print(wald_result)

"""## Breusch-Pagan Test for Heteroskedasticity"""

# Perform Breusch-Pagan test
residuals = fe_results.resids
independent_vars_bp_test = independent_vars.copy()
independent_vars_bp_test["constant"] = 1

bp_test = het_breuschpagan(resid=residuals, exog_het=independent_vars_bp_test, robust=True)
labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print(dict(zip(labels, bp_test)))


In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from linearmodels.panel import PanelOLS, RandomEffects
from statsmodels.api import add_constant
from linearmodels.panel import compare
import numpy as np
import scipy.stats
from linearmodels.panel import FirstDifferenceOLS
from statsmodels.stats.outliers_influence import variance_inflation_factor
# import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
from linearmodels.panel import PanelOLS
# from sklearn.preprocessing import StandardScaler
from statsmodels.stats.diagnostic import het_breuschpagan
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
project_folder=os.getcwd()
project_folder

'C:\\Users\\erott\\Documents\\Masterarbeit'

# SCRIPT

## DATA READING

In [ ]:
# data from Excel
filename3 = os.path.join(project_folder, "Regression data", "panel_regression_all.csv")
data3 = pd.read_csv(os.path.join(project_folder, filename2))
#'Country' as entity and 'Year' as time
data3 = data3.set_index(['Country', 'Year'])
data3=data3.dropna(axis=1, how="all")
data3.head(20)

Import    Fit    SD Fit  Duration  SD Duration  GDP/capita  \
Country Year                                                                  
Austria 2005    1436678  0.672  0.299658        13     0.000000    38417.46   
        2006   41083867  0.678  0.247408        13     0.000000    40669.33   
        2007   20635660  0.521  0.060136        13     0.000000    46915.34   
        2008   28782325  0.559  0.066811        13     0.000000    51919.98   
        2009   18342478  0.530  0.069027        13     0.000000    48153.32   
        2010   66300152  0.434  0.078703        13     0.000000    46903.76   
        2011   54727543  0.438  0.050610        13     0.000000    51442.28   
        2012   96135630  0.338  0.078276        13     0.000000    48564.92   
        2013   66585288  0.230  0.101873        13     0.000000    50731.13   
        2014   38423535  0.000  0.162951         0     5.200000    51786.38   
        2015   33389376  0.000  0.177098         0     6.368673    44195.82   
        2016   22015038  0.000  0.143312         0     6.368673    45307.59   
        2017   20463038  0.000  0.092200         0     5.200000    47429.16   
        2018   26233065  0.000  0.000000         0     0.000000    51466.56   
        2019   63165749  0.000  0.000000         0     0.000000    50067.59   
Belgium 2005   38712939  0.000  0.000000         0     0.000000    36809.70   
        2006  175801095  0.000  0.000000         0     0.000000    38705.11   
        2007  182750887  0.000  0.000000         0     0.000000    44319.17   
        2008  405533516  0.000  0.000000         0     0.000000    48303.40   
        2009  606359750  0.000  0.000000         0     0.000000    44760.29   

                 Area  Energy dependency  Energy Consumption  \
Country Year                                                   
Austria 2005  82520.0             71.760              425.46   
        2006  82520.0             72.248              421.01   
        2007  82520.0             68.475              411.99   
        2008  82520.0             68.738              420.01   
        2009  82520.0             65.122              406.84   
        2010  82520.0             62.782              420.16   
        2011  82520.0             69.981              396.07   
        2012  82520.0             63.785              414.52   
        2013  82520.0             61.260              411.20   
        2014  82520.0             65.625              394.80   
        2015  82520.0             60.379              394.84   
        2016  82520.0             62.102              407.98   
        2017  82520.0             63.930              417.20   
        2018  82520.0             64.223              408.31   
        2019  82520.0             71.601              429.03   
Belgium 2005  30280.0             79.912              756.12   
        2006  30280.0             79.486              758.24   
        2007  30280.0             76.882              763.47   
        2008  30280.0             80.846              775.73   
        2009  30280.0             75.921              715.85   

              Electricity Price  Annual Solar Capacity Addition  \
Country Year                                                      
Austria 2005           0.119000                            -6.0   
        2006           0.120775                             1.4   
        2007           0.138875                             1.9   
        2008           0.152825                             5.9   
        2009           0.165825                            18.8   
        2010           0.165050                            39.9   
        2011           0.166375                            85.3   
        2012           0.166500                           163.4   
        2013           0.169150                           288.5   
        2014           0.164525                           159.3   
        2015           0.162475                           151.9  

In [ ]:
data3.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 360 entries, ('Austria', 2005) to ('Sweden', 2019)
Data columns (total 17 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Import                           360 non-null    int64  
 1   Fit                              360 non-null    float64
 2   SD Fit                           360 non-null    float64
 3   Duration                         360 non-null    int64  
 4   SD Duration                      360 non-null    float64
 5   GDP/capita                       360 non-null    float64
 6   Area                             360 non-null    float64
 7   Energy dependency                360 non-null    float64
 8   Energy Consumption               360 non-null    float64
 9   Electricity Price                360 non-null    float64
 10  Annual Solar Capacity Addition   360 non-null    float64
 11  TechAdvancement                  360 non-null    int64 

In [ ]:
# #Standardisation of data
# scaler = StandardScaler()
# numerical_data = data.select_dtypes(include=['float64', 'int64'])
# scaled_data = scaler.fit_transform(numerical_data)
# data_scaled = pd.DataFrame(scaled_data, columns=numerical_data.columns, index=data.index)
# data_scaled

In [ ]:
data3.describe().T

,count,mean,std,min,25%,50%,75%,max
Import,360.0,2.245835e+08,8.251732e+08,32139.0000,3.082600e+06,1.363522e+07,6.823544e+07,8.340234e+09
Fit,360.0,1.557222e-01,2.274972e-01,0.0000,0.000000e+00,0.000000e+00,2.232500e-01,8.300000e-01
SD Fit,360.0,7.074416e-02,8.862287e-02,0.0000,0.000000e+00,3.077162e-02,1.133280e-01,3.411033e-01
Duration,360.0,8.263889e+00,9.297307e+00,0.0000,0.000000e+00,0.000000e+00,1.800000e+01,2.500000e+01
SD Duration,360.0,2.428685e+00,3.778216e+00,0.0000,0.000000e+00,0.000000e+00,5.669694e+00,1.224745e+01
GDP/capita,360.0,2.985638e+04,1.701686e+04,3899.8300,1.469957e+04,2.361370e+04,4.554015e+04,8.084830e+04
Area,360.0,1.660186e+05,1.562397e+05,20136.4000,5.399000e+04,8.689000e+04,2.977747e+05,5.475570e+05
Energy dependency,360.0,5.191986e+01,2.310190e+01,-50.6180,3.687400e+01,5.323850e+01,7.034575e+01,9.155100e+01
Energy Consumption,360.0,7.378902e+02,9.438475e+02,40.7500,1.879700e+02,3.415450e+02,8.358800e+02,4.062380e+03
Electricity Price,360.0,1.410688e-01,4.426644e-02,0.0580,1.122375e-01,1.334125e-01,1.632688e-01,2.871750e-01


In [ ]:
#scaling of the data
data3.loc[:, ["GDP/capita", "Tech Advancement CN", "Fixed Asset Investment CN (T-1)", "Avg Wage Difference"]] = data3.loc[:, ["GDP/capita", "Tech Advancement CN", "Fixed Asset Investment CN (T-1)", "Avg Wage Difference"]].div(1000)
data3.loc[:, "Import"] = data3.loc[:, "Import"].div(1000000)
data3.head(20)

Import    Fit    SD Fit  Duration  SD Duration  GDP/capita  \
Country Year                                                                   
Austria 2005    1.436678  0.672  0.299658        13     0.000000    38.41746   
        2006   41.083867  0.678  0.247408        13     0.000000    40.66933   
        2007   20.635660  0.521  0.060136        13     0.000000    46.91534   
        2008   28.782325  0.559  0.066811        13     0.000000    51.91998   
        2009   18.342478  0.530  0.069027        13     0.000000    48.15332   
        2010   66.300152  0.434  0.078703        13     0.000000    46.90376   
        2011   54.727543  0.438  0.050610        13     0.000000    51.44228   
        2012   96.135630  0.338  0.078276        13     0.000000    48.56492   
        2013   66.585288  0.230  0.101873        13     0.000000    50.73113   
        2014   38.423535  0.000  0.162951         0     5.200000    51.78638   
        2015   33.389376  0.000  0.177098         0     6.368673    44.19582   
        2016   22.015038  0.000  0.143312         0     6.368673    45.30759   
        2017   20.463038  0.000  0.092200         0     5.200000    47.42916   
        2018   26.233065  0.000  0.000000         0     0.000000    51.46656   
        2019   63.165749  0.000  0.000000         0     0.000000    50.06759   
Belgium 2005   38.712939  0.000  0.000000         0     0.000000    36.80970   
        2006  175.801095  0.000  0.000000         0     0.000000    38.70511   
        2007  182.750887  0.000  0.000000         0     0.000000    44.31917   
        2008  405.533516  0.000  0.000000         0     0.000000    48.30340   
        2009  606.359750  0.000  0.000000         0     0.000000    44.76029   

                 Area  Energy dependency  Energy Consumption  \
Country Year                                                   
Austria 2005  82520.0             71.760              425.46   
        2006  82520.0             72.248              421.01   
        2007  82520.0             68.475              411.99   
        2008  82520.0             68.738              420.01   
        2009  82520.0             65.122              406.84   
        2010  82520.0             62.782              420.16   
        2011  82520.0             69.981              396.07   
        2012  82520.0             63.785              414.52   
        2013  82520.0             61.260              411.20   
        2014  82520.0             65.625              394.80   
        2015  82520.0             60.379              394.84   
        2016  82520.0             62.102              407.98   
        2017  82520.0             63.930              417.20   
        2018  82520.0             64.223              408.31   
        2019  82520.0             71.601              429.03   
Belgium 2005  30280.0             79.912              756.12   
        2006  30280.0             79.486              758.24   
        2007  30280.0             76.882              763.47   
        2008  30280.0             80.846              775.73   
        2009  30280.0             75.921              715.85   

              Electricity Price  Annual Solar Capacity Addition  \
Country Year                                                      
Austria 2005           0.119000                            -6.0   
        2006           0.120775                             1.4   
        2007           0.138875                             1.9   
        2008           0.152825                             5.9   
        2009           0.165825                            18.8   
        2010           0.165050                            39.9   
        2011           0.166375                            85.3   
        2012           0.166500                           163.4   
        2013           0.169150                           288.5   
        2014           0.164525                           159.3   
        2015           0.162475             

In [ ]:
data3.describe().T

,count,mean,std,min,25%,50%,75%,max
Import,360.0,224.583454,825.173238,0.032139,3.082600,13.635224,68.235437,8340.234422
Fit,360.0,0.155722,0.227497,0.000000,0.000000,0.000000,0.223250,0.830000
SD Fit,360.0,0.070744,0.088623,0.000000,0.000000,0.030772,0.113328,0.341103
Duration,360.0,8.263889,9.297307,0.000000,0.000000,0.000000,18.000000,25.000000
SD Duration,360.0,2.428685,3.778216,0.000000,0.000000,0.000000,5.669694,12.247449
GDP/capita,360.0,29.856376,17.016865,3.899830,14.699565,23.613700,45.540148,80.848300
Area,360.0,166018.638083,156239.652736,20136.400000,53990.000000,86890.000000,297774.675000,547557.000000
Energy dependency,360.0,51.919858,23.101903,-50.618000,36.874000,53.238500,70.345750,91.551000
Energy Consumption,360.0,737.890194,943.847525,40.750000,187.970000,341.545000,835.880000,4062.380000
Electricity Price,360.0,0.141069,0.044266,0.058000,0.112237,0.133412,0.163269,0.287175


In [ ]:
# dependent, independent and control variables
data3["constant"]=1
dependent_var = data3['Import'].to_frame()
independent_vars = data3.iloc[:,1:]
dependent_var.shape, independent_vars.shape

((360, 1), (360, 17))

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(independent_vars.values, i) for i in range(independent_vars.shape[1])]
vif["features"] = independent_vars.columns
print("Variance Inflation Factors (VIF):")
vif

Variance Inflation Factors (VIF):


,VIF Factor,features
0,3.989572,Fit
1,2.496832,SD Fit
2,3.089967,Duration
3,1.721816,SD Duration
4,5.521702,GDP/capita
5,2.940381,Area
6,1.418883,Energy dependency
7,6.199970,Energy Consumption
8,2.539037,Electricity Price
9,2.457971,Annual Solar Capacity Addition


In [ ]:
correlation_matrix = data3.corr()
print(correlation_matrix)

                                   Import       Fit    SD Fit  Duration  \
Import                           1.000000  0.266253  0.080623  0.270845   
Fit                              0.266253  1.000000  0.559212  0.774316   
SD Fit                           0.080623  0.559212  1.000000  0.445053   
Duration                         0.270845  0.774316  0.445053  1.000000   
SD Duration                     -0.122346  0.009618  0.482366 -0.009277   
GDP/capita                       0.166235  0.013715 -0.067455 -0.010936   
Area                             0.240717  0.099172  0.019250  0.045788   
Energy dependency                0.121655  0.194650  0.157446  0.022402   
Energy Consumption               0.522634  0.292713  0.228120  0.281099   
Electricity Price                0.230993  0.001874  0.034808  0.095556   
Annual Solar Capacity Addition   0.856715  0.177242  0.049165  0.219916   
TechAdvancement                  0.794065  0.264630  0.128420  0.277321   
Tech Advancement CN      

In [ ]:
# Verify column names
print(independent_vars.columns)

# Drop the column if it exists
independent_vars = independent_vars.drop(columns=["constant","Fixed Asset Investment CN (T-1)","Avg Wage Difference"], errors='ignore')

# Verify the result
print(independent_vars.columns)

Index(['Fit', 'SD Fit', 'Duration', 'SD Duration', 'GDP/capita', 'Area',
       'Energy dependency', 'Energy Consumption', 'Electricity Price',
       'Annual Solar Capacity Addition', 'TechAdvancement ',
       'Tech Advancement CN', 'Trade Policies EU',
       'Fixed Asset Investment CN (T-1)', 'Avg Wage Difference',
       'Environ. St. Difference', 'constant'],
      dtype='object')
Index(['Fit', 'SD Fit', 'Duration', 'SD Duration', 'GDP/capita', 'Area',
       'Energy dependency', 'Energy Consumption', 'Electricity Price',
       'Annual Solar Capacity Addition', 'TechAdvancement ',
       'Tech Advancement CN', 'Trade Policies EU', 'Environ. St. Difference'],
      dtype='object')


In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(independent_vars.values, i) for i in range(independent_vars.shape[1])]
vif["features"] = independent_vars.columns
print("Variance Inflation Factors (VIF):")
vif

Variance Inflation Factors (VIF):


,VIF Factor,features
0,5.566317,Fit
1,3.927589,SD Fit
2,5.176692,Duration
3,2.367926,SD Duration
4,7.814158,GDP/capita
5,5.037430,Area
6,5.399971,Energy dependency
7,7.613314,Energy Consumption
8,18.990605,Electricity Price
9,2.592378,Annual Solar Capacity Addition


## MODELS

### FE MODEL (ENTITY ONE WAY)

In [ ]:
print(independent_vars.columns.duplicated())

[False False False False False False False False False False False False
 False False]


In [ ]:
corr_matrix = independent_vars.corr()
print(corr_matrix)

                                     Fit    SD Fit  Duration  SD Duration  \
Fit                             1.000000  0.559212  0.774316     0.009618   
SD Fit                          0.559212  1.000000  0.445053     0.482366   
Duration                        0.774316  0.445053  1.000000    -0.009277   
SD Duration                     0.009618  0.482366 -0.009277     1.000000   
GDP/capita                      0.013715 -0.067455 -0.010936    -0.165907   
Area                            0.099172  0.019250  0.045788    -0.090150   
Energy dependency               0.194650  0.157446  0.022402     0.030546   
Energy Consumption              0.292713  0.228120  0.281099    -0.054266   
Electricity Price               0.001874  0.034808  0.095556     0.071347   
Annual Solar Capacity Addition  0.177242  0.049165  0.219916    -0.081569   
TechAdvancement                 0.264630  0.128420  0.277321    -0.081921   
Tech Advancement CN            -0.364941 -0.106835 -0.258851     0.161921   

In [ ]:
def matrix_rank_check(df):
    return np.linalg.matrix_rank(df.values)

print(f"Rank of independent variables matrix: {matrix_rank_check(independent_vars)}")
print(f"Number of columns in independent variables: {independent_vars.shape[1]}")

Rank of independent variables matrix: 14
Number of columns in independent variables: 14


In [ ]:
print(corr_matrix)

                                     Fit    SD Fit  Duration  SD Duration  \
Fit                             1.000000  0.559212  0.774316     0.009618   
SD Fit                          0.559212  1.000000  0.445053     0.482366   
Duration                        0.774316  0.445053  1.000000    -0.009277   
SD Duration                     0.009618  0.482366 -0.009277     1.000000   
GDP/capita                      0.013715 -0.067455 -0.010936    -0.165907   
Area                            0.099172  0.019250  0.045788    -0.090150   
Energy dependency               0.194650  0.157446  0.022402     0.030546   
Energy Consumption              0.292713  0.228120  0.281099    -0.054266   
Annual Solar Capacity Addition  0.177242  0.049165  0.219916    -0.081569   
TechAdvancement                 0.264630  0.128420  0.277321    -0.081921   
Tech Advancement CN            -0.364941 -0.106835 -0.258851     0.161921   
Trade Policies EU              -0.300296 -0.017715 -0.140710     0.099049   

In [ ]:
# Fixed Effects Model
fe_model = PanelOLS(dependent_var, independent_vars, entity_effects=True)
fe_results = fe_model.fit()

print("Fixed Effects Model Results:")
fe_results

AbsorbingEffectError: 
The model cannot be estimated. The included effects have fully absorbed
one or more of the variables. This occurs when one or more of the dependent
variable is perfectly explained using the effects included in the model.

The following variables or variable combinations have been fully absorbed
or have become perfectly collinear after effects are removed:

          Fit, SD Fit, Duration, SD Duration, Area, Electricity Price, Tech Advancement CN

Set drop_absorbed=True to automatically drop absorbed variables.


### FE MODEL (TWO-WAY)

In [ ]:
#two-way fixed effects model
two_model = PanelOLS(dependent_var, independent_vars, entity_effects=True, time_effects=True,  drop_absorbed = True)
results = two_model.fit()
results

Dep. Variable:,Import,R-squared:,0.7841
Estimator:,PanelOLS,R-squared (Between):,-35.715
No. Observations:,360,R-squared (Within):,0.7766
Date:,"Mon, Aug 05 2024",R-squared (Overall):,-14.751
Time:,15:35:32,Log-likelihood,-2541.7
Cov. Estimator:,Unadjusted,,
,,F-statistic:,113.34
Entities:,24,P-value,0.0000
Avg Obs:,15.000,Distribution:,"F(10,312)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,113.34


### RANDOM EFFECTS

In [ ]:
# Random Effects Model
re_model = RandomEffects(dependent_var, independent_vars)
re_results = re_model.fit()

print("\nRandom Effects Model Results:")
re_results


Random Effects Model Results:


Dep. Variable:,Import,R-squared:,0.8376
Estimator:,RandomEffects,R-squared (Between):,0.9567
No. Observations:,360,R-squared (Within):,0.7550
Date:,"Wed, Aug 07 2024",R-squared (Overall):,0.8408
Time:,14:49:05,Log-likelihood,-2608.8
Cov. Estimator:,Unadjusted,,
,,F-statistic:,127.44
Entities:,24,P-value,0.0000
Avg Obs:,15.000,Distribution:,"F(14,346)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,127.44


In [ ]:
#Hausmann Test
comparison = compare({"Fixed": fe_results, "Random": re_results})
comparison

,Fixed,Random
Dep. Variable,Import,Import
Estimator,PanelOLS,RandomEffects
No. Observations,360,360
Cov. Est.,Unadjusted,Unadjusted
R-squared,0.7987,0.8375
R-Squared (Within),0.7987,0.7484
R-Squared (Between),-32.646,0.9583
R-Squared (Overall),-13.433,0.8377
F-statistic,162.65,226.76
P-value (F-stat),0.0000,0.0000


### Durbin-Wu-Hausman test

In [ ]:
import numpy as np
from statsmodels.regression.linear_model import OLS
from scipy.stats import chi2

def durbin_wu_hausman_test(ols_fixed, ols_random):
    """
    Perform the Durbin-Wu-Hausman test for endogeneity.

    Parameters:
    ols_fixed : OLS
        Fixed-effects regression results
    ols_random : OLS
        Random-effects regression results

    Returns:
    chi_squared_stat : float
        The chi-squared test statistic
    p_value : float
        The p-value of the test
    """
    # Residuals from fixed effects model
    u_fe = ols_fixed.resids

    # Residuals from random effects model
    u_re = ols_random.resids

    # Calculate the test statistic
    dw_hausman_stat = np.sum((u_fe - u_re)**2) / np.sum(u_fe**2)

    df = fe_results.df_model - re_results.df_model

    # Compute the p-value
    p_value = 1 - chi2.cdf(dw_hausman_stat, df)

    return dw_hausman_stat, p_value

# Example usage:
# Assuming you have panel data stored in a DataFrame called 'data'
# with columns 'dependent_var', 'independent_var1', 'independent_var2', etc.
# and a variable 'id' for individual identifiers and 'time' for time identifiers.


# Perform the Durbin-Wu-Hausman test
dw_hausman_stat, p_value = durbin_wu_hausman_test(fe_results, re_results)

print("Durbin-Wu-Hausman Test Statistic:", dw_hausman_stat)
print("P-value:", p_value)

Durbin-Wu-Hausman Test Statistic: 0.4026918013803389
P-value: 1.0


In [ ]:
fe_results

Dep. Variable:,Import,R-squared:,0.7987
Estimator:,PanelOLS,R-squared (Between):,-32.646
No. Observations:,360,R-squared (Within):,0.7987
Date:,"Fri, Jul 12 2024",R-squared (Overall):,-13.433
Time:,18:09:46,Log-likelihood,-2552.5
Cov. Estimator:,Unadjusted,,
,,F-statistic:,162.65
Entities:,24,P-value,0.0000
Avg Obs:,15.000,Distribution:,"F(8,328)"
Min Obs:,15.000,,
Max Obs:,15.000,F-statistic (robust):,162.65


In [ ]:
#Wald Test: H0 is that all the regressors are not associated (no effect) with the dependent variables, H0 model has only the intercept (unconditional average)
wald_result = fe_results.wald_test(formula="Fit=0, `GDP/capita`=0, `Energy Consumption`=0, `Annual Solar Capacity Addition`=0, `Tech Advancement CN`=0, `TechAdvancement `=0, `Trade Policies EU`=0, `Environ. St. Difference`=0")
wald_result

Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 1301.1621
P-value: 0.0000
Distributed: chi2(8)
WaldTestStatistic, id: 0x1db5869e2a0

In [ ]:
# Perform the Breusch-Pagan test
residuals = fe_results.resids
independent_vars_bp_test = independent_vars.copy()
independent_vars_bp_test["constant"]=1
independent_vars_bp_test

bp_test = het_breuschpagan(resid=residuals, exog_het=independent_vars_bp_test, robust=True)
labels = ['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value']
print(dict(zip(labels, bp_test)))

{'Lagrange multiplier statistic': 71.49189800125988, 'p-value': 2.4785064769839843e-12, 'f-value': 10.872162698637055, 'f p-value': 1.0472299529299159e-13}
